In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle as w


Goal:

Use data from 1923-01-01 to 2013-09-01 of average monthly temperatures to forecast the next 12 months' average temps in New York city.

Date range of predicitions:
2013-10-01 to 2014-10-01

In [2]:
#Acquiring and Prepping Data
df = w.wrangle_new_york()

In [3]:
# Splitting Data into Train, Validate, and Test sets
train, val, test = w.split_new_york(df)

Next Steps

Engineer features for rolling averages at various time windows to see if it reduces RMSE